# LinkedIn Jobs List

#### Importing necessary libraries

###### Important 

In [ ]:
from selenium import webdriver
import time
import pandas as pd 
import os
import requests
from bs4 import BeautifulSoup

In [ ]:
from selenium.webdriver.support import select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

###### Load Web Driver and Get the URL

###### Loading the driver

In [ ]:
driver=webdriver.Chrome(executable_path=r"C:/Users/chitr/Downloads/chromedriver_win32/chromedriver.exe")
driver.get("https://www.linkedin.com")

###### Signing in

In [ ]:
username = driver.find_element(By.ID, "session_key")
password = driver.find_element(By.ID, "session_password")
username.send_keys(" ")
password.send_keys(" ")
driver.implicitly_wait(5)

In [ ]:
login_button=driver.find_element(By.CLASS_NAME, "sign-in-form__submit-button")
login_button.click()

#### Collecting the links for all pages

In [ ]:
pages_url=[]

limit=0

while limit<1000:
    l="https://www.linkedin.com/jobs/search/?currentJobId=3443320060&geoId=102713980&keywords=Data%20analyst&location=India&originalSubdomain=in&refresh=true&start=" +str(limit)
    pages_url.append(l)
    limit+=25

pages_url

In [ ]:
len(pages_url)

#### Extracting the required data

###### Scraping through multiple pages and job postings

In [ ]:
in_text=[]
Link_list=[]
Soup_obj_list=[]

for i in range(len(pages_url)):
    driver.get(pages_url[i])
    driver.implicitly_wait(3)
    
    webpage=requests.get(pages_url[i]).text
    
    Soup_obj_list.append(BeautifulSoup(webpage))
    
    for link in Soup_obj_list[i].find_all("a"):
        Link_list.append(link.get("href"))
        in_text.append("Inner Text: {}".format(link.text.strip()))

In [ ]:
Link_list

###### Segregating the relevant links for job posting

In [ ]:
Job_Links=[]
Company_Links=[]

for i in range(len(Link_list)):
    if 'jobs/view' in str(Link_list[i]):
        Job_Links.append(Link_list[i])
    elif '/company/' in str(Link_list[i]):
        Company_Links.append(Link_list[i])
    else:
        continue

In [ ]:
len(Link_list),len(Job_Links), len(Company_Links)

###### Scraping more of relevant data

In [ ]:
Job_designation=[]
Company_name=[]
City=[]
Applicants_count=[]
Job_level=[]
E_count_ind=[]
Emp_count=[]
Linkedin_emp=[]
Company_followers=[]

In [ ]:
count=0

for l in range(0,len(Job_Links)):
    
    count+=1
    print(count)
    
    driver.get(Job_Links[l])
    driver.implicitly_wait(3)
    
    #job designation
    try:
        job=driver.find_elementse(By.CLASS_NAME, "jobs-unified-top-card__job-title")
        Job_designation.append(job[0].text)
    except:
        Job_designation.append('NULL')
    
    #company name
    try:
        company_name=driver.find_elements(By.CLASS_NAME, "jobs-unified-top-card__company-name")
        Company_name.append(company_name[0].text)
    except:
        Company_name.append('NULL')
    
    #City
    try:
        locationname=driver.find_elements(By.CLASS_NAME, "jobs-unified-top-card__bullet")
        City.append(locationname[0].text)
    except:
        City.append('NULL')
     
    #Count of applications
    try:
        applicants_count=driver.find_elements(By.CLASS_NAME, "jobs-unified-top-card__applicant-count")
        Applicants_count.append(applicants_count[0].text)
    except:
        Applicants_count.append('NULL')
     
    #Job level and Emp count
    try:
        job_nature=driver.find_elements(By.CLASS_NAME, "jobs-unified-top-card__job-insight")
        Job_level.append(job_nature[0].text)
        E_count_ind.append(job_nature[1].text)
    except:
        Job_level.append('NULL')
        E_count_ind.append('NULL')
     
    #Emp count and No of employees
    try:
        emp_count_Linkedin=driver.find_elements(By.CLASS_NAME, "jobs-company__inline-information")
        Emp_count.append(emp_count_onLinkedin[0].text)
        Linkedin_emp.append(emp_count_onLinkedin[1].text)
        
    except:
        Emp_count.append('NULL')
        Linkedin_emp.append('NULL')
        
     #No of followers   
    try:
        followers=driver.find_elements(By.CLASS_NAME, "artdeco-entity-lockup__subtitle")
        Company_followers.append(followers[0].text)
    except:
        Company_followers.append('NULL')
    
    time.sleep(5)

#### Conveting into dataframe

In [ ]:
col=["designation","Company","location","Applicant_count","Jobnature_level","Empcount_Ctype","Empcount","Emp_onLinkedin","Companyfollowers"]

df=pd.DataFrame({"Designation":Job_designation,"Company":Company_name,"Location":City,"Applicant_count":Applicants_count,"Jobnature":Job_level,"Empcount_Ctype":E_count_ind,"Employees_count":Emp_count,"Employees_on_Linkedin":Linkedin_emp,"Company_Followers":Company_followers})

In [ ]:
df.head()

#### Converting to excel file

In [ ]:
df.to_excel('Joblist.xlsx')